In [4]:
# 1. IMPROVED AGENT RESULT PARSING
# Instead of text parsing, use structured JSON responses

class StructuredAgentResponse:
    """Force agents to return structured data"""
    
    AGENT_RESPONSE_SCHEMA = """
    You must return your analysis in the following JSON format:
    {
        "recommendation": "BUY|HOLD|SELL",
        "confidence_score": 0.0-1.0,
        "price_target": float,
        "stop_loss": float,
        "key_metrics": {
            "metric_name": value
        },
        "risks": ["risk1", "risk2"],
        "opportunities": ["opp1", "opp2"],
        "reasoning": "detailed explanation"
    }
    """
    
    @staticmethod
    def parse_agent_response(response_text: str) -> dict:
        """Extract JSON from agent response"""
        import re
        import json
        
        # Find JSON block in response
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except json.JSONDecodeError:
                pass
        
        # Fallback to text parsing if JSON fails
        return StructuredAgentResponse._fallback_parse(response_text)

In [5]:
# 2. AGENT MEMORY & LEARNING SYSTEM
class AgentMemorySystem:
    """Enable agents to learn from past analyses"""
    
    def __init__(self, db_path: str = "agent_memory.db"):
        import sqlite3
        self.conn = sqlite3.connect(db_path)
        self._initialize_db()
    
    def _initialize_db(self):
        """Create tables for storing agent knowledge"""
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS analysis_history (
                id INTEGER PRIMARY KEY,
                ticker TEXT,
                agent_name TEXT,
                recommendation TEXT,
                confidence REAL,
                price_at_recommendation REAL,
                timestamp DATETIME,
                outcome_30d REAL,
                outcome_90d REAL
            )
        """)
        
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS agent_patterns (
                agent_name TEXT,
                pattern_type TEXT,
                pattern_data JSON,
                success_rate REAL,
                sample_count INTEGER
            )
        """)
    
    def store_analysis(self, agent_name: str, ticker: str, result: dict):
        """Store analysis for future learning"""
        self.conn.execute("""
            INSERT INTO analysis_history 
            (ticker, agent_name, recommendation, confidence, price_at_recommendation, timestamp)
            VALUES (?, ?, ?, ?, ?, datetime('now'))
        """, (ticker, agent_name, result['recommendation'], 
              result['confidence'], result['current_price']))
        self.conn.commit()
    
    def get_agent_performance(self, agent_name: str) -> dict:
        """Retrieve historical performance metrics"""
        cursor = self.conn.execute("""
            SELECT 
                COUNT(*) as total,
                AVG(CASE WHEN outcome_30d > 0 THEN 1 ELSE 0 END) as success_rate_30d,
                AVG(confidence) as avg_confidence,
                AVG(ABS(confidence - ABS(outcome_30d/price_at_recommendation))) as calibration_error
            FROM analysis_history
            WHERE agent_name = ? AND outcome_30d IS NOT NULL
        """, (agent_name,))
        
        return dict(cursor.fetchone())


In [6]:
# 3. REAL-TIME DATA STREAMING
class RealTimeDataManager:
    """Manage real-time data feeds for agents"""
    
    def __init__(self):
        self.websocket_connections = {}
        self.data_buffers = {}
    
    async def connect_market_data(self, ticker: str):
        """Establish WebSocket connection for real-time data"""
        # In production, connect to actual market data providers
        # For demo, simulate with periodic updates
        import asyncio
        import random
        
        async def simulate_price_stream():
            base_price = 100
            while True:
                yield {
                    'ticker': ticker,
                    'price': base_price * (1 + random.uniform(-0.02, 0.02)),
                    'volume': random.randint(1000000, 5000000),
                    'timestamp': datetime.now().isoformat()
                }
                await asyncio.sleep(1)
        
        self.data_buffers[ticker] = simulate_price_stream()
    
    async def get_latest_data(self, ticker: str, window_seconds: int = 60):
        """Get recent data window for analysis"""
        if ticker not in self.data_buffers:
            await self.connect_market_data(ticker)
        
        data_points = []
        async for data in self.data_buffers[ticker]:
            data_points.append(data)
            if len(data_points) >= window_seconds:
                break
        
        return data_points


In [8]:
pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# 5. BACKTESTING & VALIDATION SYSTEM
import yfinance as yf
import pandas as pd

class BacktestingEngine:
    """Validate agent recommendations with historical data"""
    
    def __init__(self):
        self.historical_data = {}
    
    async def backtest_strategy(self, ticker: str, strategy: dict, 
                                start_date: str, end_date: str) -> dict:
        """Backtest investment strategy"""
        import yfinance as yf
        import pandas as pd
        
        # Get historical data
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        # Simulate trades based on strategy
        positions = []
        cash = 100000  # Starting capital
        shares = 0
        
        for date, row in hist.iterrows():
            # Simplified backtesting logic
            signal = self._generate_signal(row, strategy)
            
            if signal == 'BUY' and cash > row['Close']:
                shares_to_buy = int(cash * 0.95 / row['Close'])  # Use 95% of cash
                if shares_to_buy > 0:
                    positions.append({
                        'date': date,
                        'action': 'BUY',
                        'price': row['Close'],
                        'shares': shares_to_buy
                    })
                    cash -= shares_to_buy * row['Close']
                    shares += shares_to_buy
            
            elif signal == 'SELL' and shares > 0:
                positions.append({
                    'date': date,
                    'action': 'SELL',
                    'price': row['Close'],
                    'shares': shares
                })
                cash += shares * row['Close']
                shares = 0
        
        # Calculate performance metrics
        final_value = cash + shares * hist['Close'].iloc[-1]
        total_return = (final_value - 100000) / 100000
        
        # Calculate Sharpe ratio
        if positions:
            returns = pd.Series([p['price'] for p in positions if p['action'] == 'SELL']).pct_change()
            sharpe = returns.mean() / returns.std() * (252 ** 0.5) if len(returns) > 1 else 0
        else:
            sharpe = 0
        
        return {
            'total_return': total_return,
            'final_value': final_value,
            'num_trades': len(positions),
            'sharpe_ratio': sharpe,
            'positions': positions
        }
    
    def _generate_signal(self, row: pd.Series, strategy: dict) -> str:
        """Generate trading signal based on strategy"""
        # Simplified signal generation
        if row['Close'] > row['Open'] * 1.02:  # 2% gain
            return 'SELL'
        elif row['Close'] < row['Open'] * 0.98:  # 2% loss
            return 'BUY'
        return 'HOLD'


In [10]:
# 6. EXPLAINABLE AI LAYER
class ExplainabilityEngine:
    """Make agent decisions transparent and explainable"""
    
    def __init__(self):
        self.explanation_templates = {
            'fundamental': "Based on P/E ratio of {pe:.1f} (vs industry avg {ind_pe:.1f}), "
                          "profit margin of {margin:.1%}, and debt/equity of {de:.2f}",
            'technical': "Price is {price_vs_ma:.1%} above 50-day MA, RSI at {rsi:.1f}, "
                        "MACD shows {macd_signal} signal",
            'sentiment': "News sentiment score: {sentiment:.2f}, {num_articles} articles analyzed, "
                        "dominant theme: {theme}"
        }
    
    def generate_explanation(self, agent_name: str, metrics: dict) -> str:
        """Generate human-readable explanation for agent decision"""
        template = self.explanation_templates.get(agent_name, "")
        try:
            return template.format(**metrics)
        except KeyError:
            return f"Agent {agent_name} recommendation based on proprietary analysis"
    
    def create_decision_tree(self, results: dict) -> dict:
        """Create decision tree showing how final recommendation was reached"""
        tree = {
            'root': 'Final Recommendation',
            'branches': []
        }
        
        for agent_name, result in results.items():
            branch = {
                'agent': agent_name,
                'recommendation': result.get('recommendation'),
                'confidence': result.get('confidence_score'),
                'key_factors': result.get('key_metrics', {}),
                'weight': result.get('weight', 0.25)
            }
            tree['branches'].append(branch)
        
        return tree

In [13]:
# 7. RISK MANAGEMENT FRAMEWORK
class RiskManagementSystem:
    """Advanced risk management and position sizing"""
    
    def __init__(self, risk_tolerance: str = 'moderate'):
        self.risk_profiles = {
            'conservative': {'max_position': 0.05, 'stop_loss': 0.05, 'var_limit': 0.02},
            'moderate': {'max_position': 0.10, 'stop_loss': 0.08, 'var_limit': 0.05},
            'aggressive': {'max_position': 0.20, 'stop_loss': 0.15, 'var_limit': 0.10}
        }
        self.profile = self.risk_profiles[risk_tolerance]
    
    def calculate_position_size(self, portfolio_value: float, ticker_analysis: dict) -> dict:
        """Calculate optimal position size based on Kelly Criterion and risk limits"""
        confidence = ticker_analysis.get('confidence_score', 0.5)
        expected_return = ticker_analysis.get('expected_return', 0.1)
        volatility = ticker_analysis.get('volatility', 0.2)
        
        # Kelly Criterion
        kelly_fraction = (confidence * expected_return) / (volatility ** 2)
        kelly_fraction = max(0, min(kelly_fraction, 1))  # Bound between 0 and 1
        
        # Apply risk limits
        max_position = self.profile['max_position']
        recommended_position = min(kelly_fraction * 0.25, max_position)  # Use 25% of Kelly
        
        position_value = portfolio_value * recommended_position
        
        return {
            'position_size_pct': recommended_position,
            'position_value': position_value,
            'stop_loss_pct': self.profile['stop_loss'],
            'risk_amount': position_value * self.profile['stop_loss'],
            'kelly_fraction': kelly_fraction
        }
    
    def calculate_portfolio_var(self, positions: list, confidence_level: float = 0.95) -> float:
        """Calculate Value at Risk for portfolio"""
        import numpy as np
        from scipy import stats
        
        # Simplified VaR calculation
        returns = [p.get('expected_return', 0) for p in positions]
        volatilities = [p.get('volatility', 0.2) for p in positions]
        weights = [p.get('weight', 0) for p in positions]
        
        # Portfolio metrics
        portfolio_return = np.dot(weights, returns)
        portfolio_vol = np.sqrt(np.dot(weights, np.square(volatilities)))
        
        # VaR calculation
        z_score = stats.norm.ppf(1 - confidence_level)
        var = portfolio_return + z_score * portfolio_vol
        
        return abs(var)

In [14]:
# 8. INTEGRATION WITH EXTERNAL SYSTEMS
class ExternalIntegrations:
    """Connect with trading platforms and data providers"""
    
    def __init__(self):
        self.supported_brokers = ['alpaca', 'interactive_brokers', 'td_ameritrade']
        self.data_providers = ['polygon', 'alpha_vantage', 'quandl']
    
    async def execute_trade(self, broker: str, order: dict) -> dict:
        """Execute trade through broker API"""
        if broker not in self.supported_brokers:
            raise ValueError(f"Unsupported broker: {broker}")
        
        # In production, implement actual broker APIs
        # For demo, simulate trade execution
        import random
        
        execution_price = order['limit_price'] * (1 + random.uniform(-0.001, 0.001))
        
        return {
            'order_id': f"ORD-{random.randint(100000, 999999)}",
            'status': 'filled',
            'filled_price': execution_price,
            'filled_quantity': order['quantity'],
            'timestamp': datetime.now().isoformat()
        }
    
    async def get_alternative_data(self, ticker: str, data_type: str) -> dict:
        """Fetch alternative data (satellite, web scraping, etc.)"""
        alternative_data_sources = {
            'social_sentiment': self._get_social_sentiment,
            'satellite_data': self._get_satellite_data,
            'web_traffic': self._get_web_traffic,
            'job_postings': self._get_job_postings
        }
        
        if data_type in alternative_data_sources:
            return await alternative_data_sources[data_type](ticker)
        
        return {'error': f'Unknown data type: {data_type}'}
    
    async def _get_social_sentiment(self, ticker: str) -> dict:
        """Simulate social media sentiment data"""
        import random
        return {
            'ticker': ticker,
            'reddit_mentions': random.randint(10, 1000),
            'twitter_sentiment': random.uniform(-1, 1),
            'stocktwits_bullish_pct': random.uniform(0.3, 0.7)
        }


In [ ]:
import os

# Set your API keys here
os.environ['GOOGLE_API_KEY'] = 'your-gemini-api-key-here'  # REQUIRED
os.environ['SEC_API_KEY'] = 'your-sec-api-key-here'        # Optional but recommended
os.environ['NEWS_API_KEY'] = 'your-news-api-key-here'      # Optional but recommended

# Disable Vertex AI (use Google AI Studio instead)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Verify keys are set
print("✅ Google API Key:", "Set" if os.environ.get('GOOGLE_API_KEY') else "❌ Missing")
print("✅ SEC API Key:", "Set" if os.environ.get('SEC_API_KEY') else "⚠️ Missing (some features disabled)")
print("✅ News API Key:", "Set" if os.environ.get('NEWS_API_KEY') else "⚠️ Missing (some features disabled)")